# Week 6: Probabilistic State Estimation

## Module II: Perception & Localization

### Topics Covered

- Modeling Uncertainty with Probability Distributions
- Bayes Filter Framework (Predict + Update)
- Kalman Filter for Linear Systems
- Extended Kalman Filter (EKF) for Nonlinear Systems
- Unscented Kalman Filter (UKF) for Highly Nonlinear Systems
- Sensor Fusion Applications

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Understand probabilistic representations of uncertainty (Gaussian distributions)
2. Implement the Bayes filter framework for recursive state estimation
3. Build and tune a Kalman Filter for linear state estimation problems
4. Extend to EKF for nonlinear systems using Jacobian linearization
5. Implement UKF using sigma points for better nonlinear handling
6. Apply filters to sensor fusion (GPS + IMU, LIDAR + RADAR)
7. Compare performance of different filter types on various scenarios

---

## Setup

Import required libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Additional imports as needed

## 1. Modeling Uncertainty

In autonomous systems, we never have perfect information about the world. Sensors are noisy, models are approximate, and the environment is unpredictable. Probabilistic state estimation provides a principled framework for reasoning under uncertainty.

### Sources of Uncertainty

1. **Measurement Noise**: Sensors provide imperfect observations
   - GPS accuracy: ±5-10 meters
   - LIDAR noise: ±2-3 cm
   - Camera pixel uncertainty

2. **Process Noise**: System dynamics are not perfectly modeled
   - Wind effects on vehicle motion
   - Tire slip
   - Unmodeled disturbances

3. **Model Uncertainty**: Simplified representations of reality
   - Linearization errors
   - Parameter uncertainty

### Probability Distributions

We represent uncertainty using probability distributions:

**Gaussian (Normal) Distribution:**
$$p(x) = \frac{1}{\sqrt{2\pi\sigma^2}} e^{-\frac{(x-\mu)^2}{2\sigma^2}}$$

- Mean $\mu$: Expected value
- Variance $\sigma^2$: Uncertainty

**Multivariate Gaussian:**
$$p(\mathbf{x}) = \frac{1}{\sqrt{(2\pi)^n|\boldsymbol{\Sigma}|}} e^{-\frac{1}{2}(\mathbf{x}-\boldsymbol{\mu})^T\boldsymbol{\Sigma}^{-1}(\mathbf{x}-\boldsymbol{\mu})}$$

- Mean vector $\boldsymbol{\mu}$
- Covariance matrix $\boldsymbol{\Sigma}$ (captures correlations)

In [ ]:
# Complete Kalman Filter Implementation

class KalmanFilter:
    """Linear Kalman Filter for state estimation."""
    
    def __init__(self, F, H, Q, R, x0, P0):
        """
        Initialize Kalman Filter.
        
        Args:
            F: State transition matrix (n x n)
            H: Measurement matrix (m x n)
            Q: Process noise covariance (n x n)
            R: Measurement noise covariance (m x m)
            x0: Initial state estimate (n x 1)
            P0: Initial covariance estimate (n x n)
        """
        self.F = F  # State transition
        self.H = H  # Measurement
        self.Q = Q  # Process noise
        self.R = R  # Measurement noise
        
        self.x = x0  # State estimate
        self.P = P0  # Covariance estimate
        
        self.n = F.shape[0]  # State dimension
        self.m = H.shape[0]  # Measurement dimension
    
    def predict(self, u=None):
        """
        Prediction step.
        
        Args:
            u: Control input (optional)
        """
        # State prediction: x̂⁻ = F * x̂
        self.x = self.F @ self.x
        if u is not None:
            self.x += u
        
        # Covariance prediction: P⁻ = F * P * F^T + Q
        self.P = self.F @ self.P @ self.F.T + self.Q
    
    def update(self, z):
        """
        Update step.
        
        Args:
            z: Measurement vector (m x 1)
        """
        # Innovation: y = z - H * x̂⁻
        y = z - self.H @ self.x
        
        # Innovation covariance: S = H * P⁻ * H^T + R
        S = self.H @ self.P @ self.H.T + self.R
        
        # Kalman gain: K = P⁻ * H^T * S^(-1)
        K = self.P @ self.H.T @ np.linalg.inv(S)
        
        # State update: x̂ = x̂⁻ + K * y
        self.x = self.x + K @ y
        
        # Covariance update: P = (I - K * H) * P⁻
        I = np.eye(self.n)
        self.P = (I - K @ self.H) @ self.P
    
    def get_state(self):
        """Return current state estimate."""
        return self.x.copy()
    
    def get_covariance(self):
        """Return current covariance estimate."""
        return self.P.copy()


# Demonstration: 2D Position and Velocity Tracking

np.random.seed(42)

# Simulation parameters
dt = 0.1  # Time step
n_steps = 100
t = np.arange(n_steps) * dt

# True trajectory (sinusoidal motion)
true_x = 10 * np.sin(0.5 * t)
true_vx = 10 * 0.5 * np.cos(0.5 * t)
true_y = 5 * t
true_vy = 5 * np.ones(n_steps)

# Generate noisy measurements (position only)
measurement_noise_std = 2.0
measured_x = true_x + np.random.randn(n_steps) * measurement_noise_std
measured_y = true_y + np.random.randn(n_steps) * measurement_noise_std

# Kalman Filter setup
# State: [x, vx, y, vy]
# Measurement: [x, y]

# State transition matrix (constant velocity model)
F = np.array([
    [1, dt, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, dt],
    [0, 0, 0, 1]
])

# Measurement matrix (observe position only)
H = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0]
])

# Process noise covariance
process_noise_std = 0.5
Q = np.array([
    [dt**4/4, dt**3/2, 0, 0],
    [dt**3/2, dt**2, 0, 0],
    [0, 0, dt**4/4, dt**3/2],
    [0, 0, dt**3/2, dt**2]
]) * process_noise_std**2

# Measurement noise covariance
R = np.eye(2) * measurement_noise_std**2

# Initial state and covariance
x0 = np.array([0, 0, 0, 5])  # Initial guess
P0 = np.eye(4) * 10  # High initial uncertainty

# Create filter
kf = KalmanFilter(F, H, Q, R, x0, P0)

# Run filter
estimates = []
covariances = []

for i in range(n_steps):
    # Prediction
    kf.predict()
    
    # Update with measurement
    z = np.array([measured_x[i], measured_y[i]])
    kf.update(z)
    
    # Store results
    estimates.append(kf.get_state())
    covariances.append(kf.get_covariance())

estimates = np.array(estimates)
covariances = np.array(covariances)

# Visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Plot 1: 2D trajectory
ax1 = axes[0, 0]
ax1.plot(true_x, true_y, 'g-', linewidth=3, label='True Trajectory', alpha=0.8)
ax1.scatter(measured_x, measured_y, c='red', s=30, alpha=0.5, label='Measurements', zorder=3)
ax1.plot(estimates[:, 0], estimates[:, 2], 'b-', linewidth=2, label='KF Estimate')

# Plot uncertainty ellipses at selected points
for i in range(0, n_steps, 20):
    # Extract position covariance
    P_pos = covariances[i][[0, 2]][:, [0, 2]]
    
    # Compute eigenvalues/eigenvectors for ellipse
    eigenvalues, eigenvectors = np.linalg.eigh(P_pos)
    angle = np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0])
    
    # 95% confidence ellipse (chi-square with 2 DOF: 5.991)
    width, height = 2 * np.sqrt(5.991 * eigenvalues)
    
    from matplotlib.patches import Ellipse
    ellipse = Ellipse((estimates[i, 0], estimates[i, 2]), width, height,
                      angle=np.degrees(angle), facecolor='blue', 
                      alpha=0.1, edgecolor='blue', linewidth=1)
    ax1.add_patch(ellipse)

ax1.set_xlabel('X Position (m)', fontsize=12)
ax1.set_ylabel('Y Position (m)', fontsize=12)
ax1.set_title('2D Trajectory Tracking', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.axis('equal')

# Plot 2: X position over time
ax2 = axes[0, 1]
ax2.plot(t, true_x, 'g-', linewidth=2, label='True')
ax2.scatter(t, measured_x, c='red', s=20, alpha=0.5, label='Measured')
ax2.plot(t, estimates[:, 0], 'b-', linewidth=2, label='KF Estimate')

# Uncertainty bounds (±2σ)
std_x = np.sqrt(covariances[:, 0, 0])
ax2.fill_between(t, estimates[:, 0] - 2*std_x, estimates[:, 0] + 2*std_x,
                 alpha=0.2, color='blue', label='95% Confidence')

ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('X Position (m)', fontsize=12)
ax2.set_title('X Position Tracking', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

# Plot 3: X velocity over time
ax3 = axes[0, 2]
ax3.plot(t, true_vx, 'g-', linewidth=2, label='True')
ax3.plot(t, estimates[:, 1], 'b-', linewidth=2, label='KF Estimate')

# Uncertainty bounds
std_vx = np.sqrt(covariances[:, 1, 1])
ax3.fill_between(t, estimates[:, 1] - 2*std_vx, estimates[:, 1] + 2*std_vx,
                 alpha=0.2, color='blue', label='95% Confidence')

ax3.set_xlabel('Time (s)', fontsize=12)
ax3.set_ylabel('X Velocity (m/s)', fontsize=12)
ax3.set_title('X Velocity Estimation', fontsize=14, fontweight='bold')
ax3.legend(fontsize=10)
ax3.grid(True, alpha=0.3)

# Plot 4: Y position over time
ax4 = axes[1, 0]
ax4.plot(t, true_y, 'g-', linewidth=2, label='True')
ax4.scatter(t, measured_y, c='red', s=20, alpha=0.5, label='Measured')
ax4.plot(t, estimates[:, 2], 'b-', linewidth=2, label='KF Estimate')

# Uncertainty bounds
std_y = np.sqrt(covariances[:, 2, 2])
ax4.fill_between(t, estimates[:, 2] - 2*std_y, estimates[:, 2] + 2*std_y,
                 alpha=0.2, color='blue', label='95% Confidence')

ax4.set_xlabel('Time (s)', fontsize=12)
ax4.set_ylabel('Y Position (m)', fontsize=12)
ax4.set_title('Y Position Tracking', fontsize=14, fontweight='bold')
ax4.legend(fontsize=10)
ax4.grid(True, alpha=0.3)

# Plot 5: Position error over time
ax5 = axes[1, 1]
position_error = np.sqrt((estimates[:, 0] - true_x)**2 + (estimates[:, 2] - true_y)**2)
measurement_error = np.sqrt((measured_x - true_x)**2 + (measured_y - true_y)**2)

ax5.plot(t, measurement_error, 'r-', linewidth=1.5, alpha=0.7, label='Measurement Error')
ax5.plot(t, position_error, 'b-', linewidth=2, label='KF Error')
ax5.axhline(np.mean(position_error), color='b', linestyle='--', linewidth=1.5,
           label=f'Mean KF: {np.mean(position_error):.2f}m')

ax5.set_xlabel('Time (s)', fontsize=12)
ax5.set_ylabel('Position Error (m)', fontsize=12)
ax5.set_title('Tracking Error Comparison', fontsize=14, fontweight='bold')
ax5.legend(fontsize=10)
ax5.grid(True, alpha=0.3)

# Plot 6: Uncertainty reduction over time
ax6 = axes[1, 2]
trace_P = np.array([np.trace(P) for P in covariances])
det_P = np.array([np.linalg.det(P) for P in covariances])

ax6_twin = ax6.twinx()
line1 = ax6.plot(t, trace_P, 'b-', linewidth=2, label='Trace(P)')
line2 = ax6_twin.plot(t, det_P, 'r-', linewidth=2, label='Det(P)')

ax6.set_xlabel('Time (s)', fontsize=12)
ax6.set_ylabel('Trace(P)', fontsize=12, color='b')
ax6_twin.set_ylabel('Det(P)', fontsize=12, color='r')
ax6.set_title('Uncertainty Reduction', fontsize=14, fontweight='bold')
ax6.tick_params(axis='y', labelcolor='b')
ax6_twin.tick_params(axis='y', labelcolor='r')

lines = line1 + line2
labels = [l.get_label() for l in lines]
ax6.legend(lines, labels, fontsize=10)
ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print statistics
print("=" * 70)
print("KALMAN FILTER PERFORMANCE")
print("=" * 70)
print(f"Simulation time: {n_steps * dt:.1f} seconds")
print(f"Time step: {dt} seconds")
print(f"Measurement noise std: {measurement_noise_std:.2f} m")
print(f"Process noise std: {process_noise_std:.2f}")
print(f"\nPosition Error:")
print(f"  Measurement RMSE: {np.sqrt(np.mean(measurement_error**2)):.3f} m")
print(f"  KF RMSE: {np.sqrt(np.mean(position_error**2)):.3f} m")
print(f"  Improvement: {(1 - np.sqrt(np.mean(position_error**2))/np.sqrt(np.mean(measurement_error**2)))*100:.1f}%")
print(f"\nVelocity Estimation (not directly measured):")
vx_error = np.abs(estimates[:, 1] - true_vx)
vy_error = np.abs(estimates[:, 3] - true_vy)
print(f"  X velocity MAE: {np.mean(vx_error):.3f} m/s")
print(f"  Y velocity MAE: {np.mean(vy_error):.3f} m/s")
print(f"\nFinal Uncertainty (trace of covariance):")
print(f"  Initial: {np.trace(P0):.3f}")
print(f"  Final: {trace_P[-1]:.3f}")
print(f"  Reduction: {(1 - trace_P[-1]/np.trace(P0))*100:.1f}%")
print("=" * 70)

## 2. Bayes Filter Framework

The **Bayes filter** is a recursive algorithm for estimating the state of a dynamic system from noisy observations.

### Recursive State Estimation

**Goal**: Estimate state $\mathbf{x}_t$ given:
- All measurements up to time $t$: $\mathbf{z}_{1:t}$
- All controls up to time $t$: $\mathbf{u}_{1:t}$

**Belief**: Probability distribution over states
$$bel(\mathbf{x}_t) = p(\mathbf{x}_t | \mathbf{z}_{1:t}, \mathbf{u}_{1:t})$$

---

### Bayes Filter Algorithm

**Two-step process**:

#### **1. Prediction Step** (Motion Update)

Predict state based on motion model:
$$\overline{bel}(\mathbf{x}_t) = \int p(\mathbf{x}_t | \mathbf{u}_t, \mathbf{x}_{t-1}) \, bel(\mathbf{x}_{t-1}) \, d\mathbf{x}_{t-1}$$

**Intuition**: Propagate belief forward using system dynamics.

#### **2. Update Step** (Measurement Update)

Correct prediction using sensor measurement:
$$bel(\mathbf{x}_t) = \eta \, p(\mathbf{z}_t | \mathbf{x}_t) \, \overline{bel}(\mathbf{x}_t)$$

Where $\eta$ is a normalization constant.

**Intuition**: Weight prediction by likelihood of measurement.

---

### Assumptions

1. **Markov Property**: Future states depend only on current state
   $$p(\mathbf{x}_t | \mathbf{x}_{0:t-1}, \mathbf{u}_{1:t}, \mathbf{z}_{1:t-1}) = p(\mathbf{x}_t | \mathbf{x}_{t-1}, \mathbf{u}_t)$$

2. **Measurement Independence**: Measurements depend only on current state
   $$p(\mathbf{z}_t | \mathbf{x}_{0:t}, \mathbf{u}_{1:t}, \mathbf{z}_{1:t-1}) = p(\mathbf{z}_t | \mathbf{x}_t)$$

---

### Kalman Filter: Gaussian Bayes Filter

**Kalman Filter** implements the Bayes filter for linear-Gaussian systems.

**Key Insight**: If:
- Initial belief is Gaussian
- Motion and measurement models are linear with Gaussian noise
- Then belief remains Gaussian at all times!

**Representation**: Only need to track mean $\boldsymbol{\mu}_t$ and covariance $\boldsymbol{\Sigma}_t$

$$bel(\mathbf{x}_t) = \mathcal{N}(\boldsymbol{\mu}_t, \boldsymbol{\Sigma}_t)$$

---

## 3. Kalman Filter (Linear Systems)

**Kalman Filter** is optimal (minimum mean squared error) for linear systems with Gaussian noise.

### System Model

**State Transition** (Motion Model):
$$\mathbf{x}_t = \mathbf{F}_t \mathbf{x}_{t-1} + \mathbf{B}_t \mathbf{u}_t + \mathbf{w}_t$$

Where:
- $\mathbf{F}_t$: State transition matrix
- $\mathbf{B}_t$: Control input matrix
- $\mathbf{w}_t \sim \mathcal{N}(\mathbf{0}, \mathbf{Q}_t)$: Process noise

**Measurement Model**:
$$\mathbf{z}_t = \mathbf{H}_t \mathbf{x}_t + \mathbf{v}_t$$

Where:
- $\mathbf{H}_t$: Measurement matrix
- $\mathbf{v}_t \sim \mathcal{N}(\mathbf{0}, \mathbf{R}_t)$: Measurement noise

---

### Algorithm

**Prediction Step**:

$$\boldsymbol{\mu}_t^- = \mathbf{F}_t \boldsymbol{\mu}_{t-1} + \mathbf{B}_t \mathbf{u}_t$$
$$\boldsymbol{\Sigma}_t^- = \mathbf{F}_t \boldsymbol{\Sigma}_{t-1} \mathbf{F}_t^T + \mathbf{Q}_t$$

**Update Step**:

**Innovation** (measurement residual):
$$\mathbf{y}_t = \mathbf{z}_t - \mathbf{H}_t \boldsymbol{\mu}_t^-$$

**Innovation covariance**:
$$\mathbf{S}_t = \mathbf{H}_t \boldsymbol{\Sigma}_t^- \mathbf{H}_t^T + \mathbf{R}_t$$

**Kalman Gain**:
$$\mathbf{K}_t = \boldsymbol{\Sigma}_t^- \mathbf{H}_t^T \mathbf{S}_t^{-1}$$

**State update**:
$$\boldsymbol{\mu}_t = \boldsymbol{\mu}_t^- + \mathbf{K}_t \mathbf{y}_t$$

**Covariance update**:
$$\boldsymbol{\Sigma}_t = (\mathbf{I} - \mathbf{K}_t \mathbf{H}_t) \boldsymbol{\Sigma}_t^-$$

---

### Example: 1D Position Tracking

**State**: $x_t$ = position
**Measurement**: $z_t$ = noisy position

**Motion model**: $x_t = x_{t-1} + v \cdot \Delta t + w_t$ (constant velocity)
**Measurement model**: $z_t = x_t + v_t$

**Matrices**:
- $F = 1$ (identity)
- $H = 1$ (direct measurement)
- $Q$ = process noise variance
- $R$ = measurement noise variance

---

### Tuning Parameters

**Process Noise $\mathbf{Q}$**:
- **High $\mathbf{Q}$**: Trust measurements more (filter responsive)
- **Low $\mathbf{Q}$**: Trust model more (filter smooth)

**Measurement Noise $\mathbf{R}$**:
- **High $\mathbf{R}$**: Trust model more (ignore noisy sensors)
- **Low $\mathbf{R}$**: Trust measurements more (follow sensors)

**Rule of thumb**: Set based on known sensor/model characteristics.

In [ ]:
# Unscented Kalman Filter Implementation

class UnscentedKalmanFilter:
    def __init__(self, n_states, initial_state, initial_cov, process_noise, measurement_noise,
                 alpha=1e-3, beta=2, kappa=0):
        """
        UKF implementation
        
        Parameters:
        - n_states: dimension of state vector
        - alpha: spread of sigma points (typically 1e-3 to 1)
        - beta: incorporates prior knowledge (2 is optimal for Gaussian)
        - kappa: secondary scaling parameter (typically 0 or 3-n)
        """
        self.n = n_states
        self.x = initial_state
        self.P = initial_cov
        self.Q = process_noise
        self.R = measurement_noise
        
        # UKF parameters
        self.alpha = alpha
        self.beta = beta
        self.kappa = kappa
        
        self.lambda_ = alpha**2 * (n_states + kappa) - n_states
        
        # Weights for mean and covariance
        self.Wm = np.zeros(2 * n_states + 1)
        self.Wc = np.zeros(2 * n_states + 1)
        
        self.Wm[0] = self.lambda_ / (n_states + self.lambda_)
        self.Wc[0] = self.lambda_ / (n_states + self.lambda_) + (1 - alpha**2 + beta)
        
        for i in range(1, 2 * n_states + 1):
            self.Wm[i] = 1.0 / (2.0 * (n_states + self.lambda_))
            self.Wc[i] = self.Wm[i]
    
    def generate_sigma_points(self, x, P):
        """Generate sigma points around x with covariance P"""
        n = len(x)
        sigma_points = np.zeros((2 * n + 1, n))
        
        # Center point
        sigma_points[0] = x
        
        # Matrix square root using Cholesky decomposition
        try:
            U = np.linalg.cholesky((n + self.lambda_) * P)
        except np.linalg.LinAlgError:
            # If Cholesky fails, use eigenvalue decomposition
            eigvals, eigvecs = np.linalg.eigh(P)
            eigvals = np.maximum(eigvals, 0)  # Ensure positive
            U = eigvecs @ np.diag(np.sqrt((n + self.lambda_) * eigvals))
        
        # Positive sigma points
        for i in range(n):
            sigma_points[i + 1] = x + U[:, i]
        
        # Negative sigma points
        for i in range(n):
            sigma_points[n + i + 1] = x - U[:, i]
        
        return sigma_points
    
    def predict(self, dt, f):
        """
        Prediction step
        f: state transition function f(x, dt) -> x_new
        """
        # Generate sigma points
        sigma_points = self.generate_sigma_points(self.x, self.P)
        
        # Propagate sigma points through motion model
        n = len(self.x)
        sigma_points_pred = np.zeros((2 * n + 1, n))
        
        for i in range(2 * n + 1):
            sigma_points_pred[i] = f(sigma_points[i], dt)
        
        # Compute predicted mean
        self.x = np.sum(self.Wm[:, np.newaxis] * sigma_points_pred, axis=0)
        
        # Compute predicted covariance
        self.P = np.zeros((n, n))
        for i in range(2 * n + 1):
            diff = sigma_points_pred[i] - self.x
            self.P += self.Wc[i] * np.outer(diff, diff)
        
        self.P += self.Q
    
    def update(self, z, h):
        """
        Update step
        z: measurement vector
        h: measurement function h(x) -> z_pred
        """
        n = len(self.x)
        m = len(z)
        
        # Generate sigma points from predicted state
        sigma_points = self.generate_sigma_points(self.x, self.P)
        
        # Propagate sigma points through measurement model
        sigma_meas = np.zeros((2 * n + 1, m))
        for i in range(2 * n + 1):
            sigma_meas[i] = h(sigma_points[i])
        
        # Predicted measurement
        z_pred = np.sum(self.Wm[:, np.newaxis] * sigma_meas, axis=0)
        
        # Innovation covariance
        S = np.zeros((m, m))
        for i in range(2 * n + 1):
            diff = sigma_meas[i] - z_pred
            S += self.Wc[i] * np.outer(diff, diff)
        S += self.R
        
        # Cross-correlation
        Pxz = np.zeros((n, m))
        for i in range(2 * n + 1):
            dx = sigma_points[i] - self.x
            dz = sigma_meas[i] - z_pred
            Pxz += self.Wc[i] * np.outer(dx, dz)
        
        # Kalman gain
        K = Pxz @ np.linalg.inv(S)
        
        # State update
        self.x = self.x + K @ (z - z_pred)
        
        # Covariance update
        self.P = self.P - K @ S @ K.T


# Comparison: UKF vs EKF on highly nonlinear system
def compare_ukf_ekf():
    np.random.seed(42)
    
    # State transition (nonlinear)
    def f(x, dt):
        """Constant turn rate and velocity model"""
        px, py, v, theta, omega = x
        if abs(omega) < 1e-6:  # Straight line motion
            return np.array([
                px + v * np.cos(theta) * dt,
                py + v * np.sin(theta) * dt,
                v,
                theta,
                omega
            ])
        else:  # Curved motion
            return np.array([
                px + v / omega * (np.sin(theta + omega * dt) - np.sin(theta)),
                py + v / omega * (-np.cos(theta + omega * dt) + np.cos(theta)),
                v,
                theta + omega * dt,
                omega
            ])
    
    # Measurement model (range-bearing)
    def h(x):
        px, py, _, _, _ = x
        r = np.sqrt(px**2 + py**2)
        bearing = np.arctan2(py, px)
        return np.array([r, bearing])
    
    # Simulation
    dt = 0.1
    num_steps = 100
    
    # True initial state: [px, py, v, theta, omega]
    x_true = np.array([0.0, 0.0, 5.0, np.pi/4, 0.1])  # Turning motion
    
    true_states = []
    measurements = []
    
    for t in range(num_steps):
        x_true = f(x_true, dt)
        true_states.append(x_true.copy())
        
        # Noisy measurement
        z_true = h(x_true)
        z_meas = z_true + np.array([np.random.randn() * 2.0, np.random.randn() * 0.1])
        measurements.append(z_meas)
    
    true_states = np.array(true_states)
    measurements = np.array(measurements)
    
    # Initialize UKF
    x0 = np.array([0.0, 0.0, 4.0, np.pi/4, 0.0])  # Initial guess
    P0 = np.diag([5.0, 5.0, 2.0, 0.5, 0.2])
    Q = np.diag([0.1, 0.1, 0.1, 0.01, 0.01])
    R = np.diag([4.0, 0.01])
    
    ukf = UnscentedKalmanFilter(5, x0.copy(), P0.copy(), Q, R)
    
    ukf_estimates = []
    
    for i in range(num_steps):
        ukf.predict(dt, f)
        ukf.update(measurements[i], h)
        ukf_estimates.append(ukf.x.copy())
    
    ukf_estimates = np.array(ukf_estimates)
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Trajectory
    axes[0].plot(true_states[:, 0], true_states[:, 1], 'g-', label='True', linewidth=2)
    axes[0].plot(ukf_estimates[:, 0], ukf_estimates[:, 1], 'b-', label='UKF', linewidth=2)
    axes[0].plot(0, 0, 'r*', markersize=15, label='Sensor')
    axes[0].set_xlabel('X Position')
    axes[0].set_ylabel('Y Position')
    axes[0].set_title('UKF Tracking (Nonlinear Turning Motion)')
    axes[0].legend()
    axes[0].grid(True)
    axes[0].axis('equal')
    
    # Position error
    position_error = np.sqrt((ukf_estimates[:, 0] - true_states[:, 0])**2 + 
                            (ukf_estimates[:, 1] - true_states[:, 1])**2)
    time = np.arange(num_steps) * dt
    
    axes[1].plot(time, position_error, 'b-', linewidth=2)
    axes[1].set_xlabel('Time (s)')
    axes[1].set_ylabel('Position Error (m)')
    axes[1].set_title('UKF Position Error')
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Mean Position Error: {np.mean(position_error):.2f} m")
    print(f"Max Position Error: {np.max(position_error):.2f} m")

compare_ukf_ekf()

## Exercises

### Exercise 1: 1D Kalman Filter for Temperature Tracking

Implement a 1D Kalman filter to estimate room temperature from noisy sensor readings.

**Given:**
- True temperature: 20°C with small random walk (±0.1°C per step)
- Sensor noise: ±1.5°C (standard deviation)
- 100 time steps

**Tasks:**
1. Implement the 1D Kalman filter
2. Plot true temperature, measurements, and estimates
3. Calculate the mean squared error
4. Experiment with different process and measurement noise values

# Exercise 3: This is left as a challenge for the student
# Hint: Reuse the implementations from sections 3-5
# Try different turning rates (omega values) to vary nonlinearity

# Here's a template to get started:

def compare_all_filters():
    """
    Compare Linear KF (will perform poorly), EKF, and UKF
    on a turning vehicle scenario
    """
    # TODO: Implement comparison
    # 1. Set up nonlinear motion model (constant turn rate)
    # 2. Initialize all three filters with same parameters
    # 3. Run them on the same measurement sequence
    # 4. Compare errors and computation time
    # 5. Visualize results
    
    pass  # Your implementation here

# Uncomment to run:
# compare_all_filters()

## References and Additional Resources

### Core Textbooks

1. **Probabilistic Robotics** by Thrun, Burgard, and Fox (2005)
   - The definitive reference for probabilistic state estimation
   - Chapters 2-3: Recursive state estimation and Bayes filters
   - Chapter 3.2: Kalman Filter
   - Chapter 3.3: Extended Kalman Filter
   - Chapter 3.4: Unscented Kalman Filter

2. **Optimal State Estimation** by Simon (2006)
   - Comprehensive treatment of Kalman filtering
   - Practical implementation considerations

3. **State Estimation for Robotics** by Barfoot (2017)
   - Modern perspective with robotics applications
   - Available online: http://asrl.utias.utoronto.ca/~tdb/bib/barfoot_ser17.pdf

### Key Papers

1. **Kalman, R. E. (1960)** - "A New Approach to Linear Filtering and Prediction Problems"
   - Original Kalman filter paper
   - Transactions of the ASME–Journal of Basic Engineering

2. **Julier, S. J., & Uhlmann, J. K. (1997)** - "New extension of the Kalman filter to nonlinear systems"
   - Introduced the Unscented Kalman Filter
   - SPIE 3068, Signal Processing, Sensor Fusion, and Target Recognition VI

### Online Resources

1. **Kalman Filter Tutorial by Greg Welch and Gary Bishop**
   - https://www.cs.unc.edu/~welch/kalman/
   - Excellent introduction with practical examples

2. **Roger Labbe's Kalman Filter Book (Jupyter Notebooks)**
   - https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python
   - Interactive Python implementations

3. **Udacity Self-Driving Car Nanodegree**
   - Extended Kalman Filter project
   - Sensor fusion with LIDAR and RADAR

### Software Libraries

1. **FilterPy** (Python)
   - pip install filterpy
   - Kalman filter implementations

2. **PyKalman** (Python)
   - pip install pykalman
   - EM algorithm for parameter estimation

3. **Robot Operating System (ROS)**
   - robot_localization package
   - Production-ready EKF/UKF implementations

### Applications in Autonomous Vehicles

1. **Localization**: GPS/IMU fusion for position estimation
2. **Object Tracking**: Multi-object tracking with RADAR/LIDAR/Camera
3. **SLAM**: Simultaneous Localization and Mapping
4. **Sensor Calibration**: Estimating sensor biases and drift

### Related Topics to Explore

1. **Particle Filters**: Non-parametric Bayes filter for multimodal distributions
2. **Information Filter**: Inverse covariance form of Kalman filter
3. **Kalman Smoother**: Batch processing for improved estimates
4. **Multi-Hypothesis Tracking**: Handling data association uncertainty
5. **Adaptive Filtering**: Online estimation of Q and R matrices

### Visualization Tools

- **Matplotlib** (used in this notebook)
- **Plotly**: Interactive visualizations
- **RViz**: Real-time robotics visualization

---

## Summary

This notebook covered the fundamentals of probabilistic state estimation:

**Key Concepts:**
- Uncertainty representation with Gaussian distributions
- Bayes filter framework (predict + update)
- Kalman Filter for linear systems
- Extended Kalman Filter for nonlinear systems (Jacobian linearization)
- Unscented Kalman Filter for highly nonlinear systems (sigma points)

**Practical Skills:**
- Implementing filters from scratch
- Tuning process and measurement noise
- Sensor fusion techniques
- Performance evaluation

**Next Steps:**
1. Explore the HTML visualizations in this repository ([kalman_ball_chase.html](kalman_ball_chase.html))
2. Implement a real-world application (e.g., GPS/IMU fusion)
3. Study Particle Filters for non-Gaussian distributions
4. Learn about SLAM and its relation to state estimation